In [5]:
%pip install sklearn
%pip transformers
%pip install huggingface
%pip install torch 
%pip install pandas
%pip install sklearn
%pip install datasets
%pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: unknown command "transformers"
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pickle

In [17]:
#Methods
def parse_html(text):
    "removes hyperlink from a piece of text"
    
    soup = BeautifulSoup(text, "html.parser")
    parsed_text = soup.get_text()
    return parsed_text


def spacy_preproc(text):

    "Removes stopwords and punctuation and"
    nlp = English()
    tokenizer = nlp.tokenizer
    refined_text = tokenizer(text)
        
    refined_text = [text for text in refined_text if not text.is_stop]
    refined_text = [text for text in refined_text if not text.is_punct]
    refined_text = [text for text in refined_text if not text.like_email]
    refined_text = [text for text in refined_text if not text.like_url]
    refined_text = [text.text for text in refined_text if not text.like_num]

    text = " ".join(refined_text[:])

    return text

def regex_preproc(text): 

    refined_text = re.sub("(\n)", " ", text) #removes python newline 
    refined_text = re.sub("\w+(\')", "" , refined_text) #removes backslash
    refined_text = re.sub("(<\W+(p >))", " ", refined_text)
    refined_text = re.sub("(\s+)", " ", refined_text) #removes whitespaces
    #refined_text = re.sub("(&amp|>+|-+)", "", refined_text)
    #refined_text = re.sub("\W*(@)", "", refined_text) #removes emails 
    #refined_text = re.sub("(\d+)", "", refined_text) #removes digits

    return refined_text
                
def preproc(text):
  """Preprocesses files"""

  #output = parse_html(text)
  temp_output = spacy_preproc(text)
  final_output = regex_preproc(temp_output)

  return final_output


def prepare_corpus(path):
    """BY date"""
    corpus = {}; temp = []; 
    for topic in os.listdir(path):
        subfolder = path + '/' + topic
        current = []
        for doc in os.listdir(subfolder):
            file = subfolder + '/' + doc
            with open(file, 'r', encoding='utf-8', errors= 'ignore') as t:
               #corpus.append((preproc(" ".join(t.readlines())),topic),)
               temp = " ".join(t.readlines())
            current.append(temp)
        corpus[topic] = current
    return corpus


def extra_long_documents(text, chunk_size = 100):

  """
  Takes in a long documents chunks them into segments of lenght 'chunk_size'
  """
  doc_length = len(text); chunks = []
  
  if doc_length > chunk_size:
    output = range(doc_length//chunk_size)

    chunks = [text[:chunk_size*n] for n in output]
    last_index = output[-1]*chunk_size

    chunks.append(text[last_index:])
  else :
    chunks.append(text)
  
    return chunks 

def extract_features(text, min_df = 0.05 , max_df = 0.4, max_features = 100, method = "Tfidf"):
    """Count represents the number of features to be chosen from tfidf while text represents text data
    vectorizer is either count_vectorizer or tfidfvectorizer"""
    
    if method.lower() == "tfidf":
        vectorizer = TfidfVectorizer(token_pattern = '[A-Za-z]+', min_df = float(min_df), max_df = float
                                     (max_df), ngram_range = (1,1), max_features = max_features)
    else:
        print("Wrong method. Choose between tfidf")

    vec = vectorizer.fit_transform(text)
    X_features = vectorizer.get_feature_names()
    print(vec.shape)
    return vec, X_features

#^\s

def accuracy(prediction, goldstandard):
    acc = 0
    for a,b in zip(prediction,goldstandard):
        if a == b:
            acc+=1
        else:pass
    return (acc/len(prediction))*100

In [8]:
#from sklearn.datasets import fetch_20newsgroups #encountered issues SSLCertificationError with this 
from datasets import load_dataset
import os
import pandas as pd
import re
from bs4 import BeautifulSoup

from spacy import tokenizer
from spacy.lang.en import English
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm


In [9]:
hyperpartisan_dataset = load_dataset("hyperpartisan_news_detection", "bypublisher") #Not stripped of headers and footers
hyperpartisan_dataset.keys()

Reusing dataset hyperpartisan_news_detection (/root/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012)


  0%|          | 0/2 [00:00<?, ?it/s]

dict_keys(['train', 'validation'])

In [10]:
#change to bypublisher because byarticle has no test set.
hyperpartisan_train = hyperpartisan_dataset['train']
hyperpartisan_valid = hyperpartisan_dataset['validation']

hyperpartisan_train = hyperpartisan_train.shuffle(seed = 32)
hyperpartisan_valid = hyperpartisan_valid.shuffle(seed = 32)

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012/cache-a00921e944b59924.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012/cache-4a5b1cc91f59ed6a.arrow


In [11]:
#If hyperpartisan use 1
#Else use 0
int_labels_train = [1 if x == True else 0 for x in hyperpartisan_train['hyperpartisan']]
int_labels_val = [1 if x == True else 0 for x in hyperpartisan_valid['hyperpartisan']]

In [12]:
#processed_train = [preproc(text) for text in hyperpartisan_train['text']]
processed_train = [regex_preproc(parse_html(text)) for text in hyperpartisan_train["text"]]
processed_test = [regex_preproc(parse_html(text)) for text in hyperpartisan_train["text"]]

In [14]:
import pickle

In [15]:
with open('processed_train.pkl', "wb") as out:
  pickle.dump(processed_train, out)

with open('processed_test.pkl', "wb") as out_2:
  pickle.dump(processed_test, out_2)


In [ ]:
vec, feat = extract_features(processed_train, method = "tfidf")
vec.shape

In [ ]:
print(feat)

In [ ]:
#Initialising multinomial naive bayes
MNB = MultinomialNB()

#explore other forms of naive bayes
MNB.fit(vec[:,:],int_labels_train)

#extract features from test data
test_vec,test_features = extract_features(hyperpartisan_valid['text'])

predicted_topic = MNB.predict(test_vec[:,:])

accuracy(predicted_topic, int_labels_val)

In [ ]:
for max_features in range(100,1000,100):
  
  for max_df in np.arange(0.1, 0.3, 0.05):


SVM

In [ ]:
svc_classifier = svm.SVC(decision_function_shape='ovo')
svc_classifier.fit(vec[:,:],int_labels_train[:])
svc_prediction = svc_classifier.predict(test_vec[:,:])

accuracy(svc_prediction, int_labels_val)